In [85]:
import pandas as pd

In [86]:
story_titles = pd.read_csv("data.csv")['title'].tolist()

Aqui é extraída a trama básica da história da api em [stories.rafaelaraujo.tech](https://stories.rafaelaraujo.tech)

In [87]:
import requests
def get_info(title):
    response = requests.get(f'https://stories.rafaelaraujo.tech/story/{title}')

    if response.status_code == 200:
      response_json = response.json()
      story = {
        'id': response_json['id'],
        'description': response_json['description'],
        'title': response_json['title']
        }
      return story
    else:
      return None

In [88]:
stories = []
for title in story_titles:
  s = get_info(title)
  if s:
    stories.append(s)

In [89]:
openai_api_key = # your-api-key

In [ ]:
!pip install openai

In [91]:
import openai

openai.api_key = openai_api_key

def generate_story(description):
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {
              "role": "system",
              "content": "Act as a creative writer"
          },
          {
              "role": "user",
              "content": f'Create a short story based on the following description: "{description}". It should have an introduction, development and ending. It should have less than 100 words.'
          }
      ]
  )

  return completion.choices[0].message.content.strip('\*').replace("'", "").replace("\n", "")

In [ ]:
for story in stories:
  text = generate_story(story['description'])
  while len(text) >= 1000:
    # ChatGPT can't count words properly, this part keeps trying until it gets an appropriate-sized story
    text = generate_story(story['description'])

  story['full_story'] = text
  print("Generated full story for " + story['title'])

Aqui, a história final gerada é salva com uma requisição POST para o endpoint [stories.rafaelaraujo.tech/story](https://stories.rafaelaraujo.tech/story)

In [93]:
headers = {'Content-Type': "application/json", 'Accept': "*/*", "Connection": "keep-alive"}
def save_story(id, full_story):
  response = requests.post(f'https://stories.rafaelaraujo.tech/story', headers=headers, json={
        "fullStory": full_story,
        "parentStoryId": id
  })

  return 1 if response.status_code == 200 else 0

In [ ]:
for story in stories:
  response = save_story(story['id'], story['full_story'])
  if response == 1:
    print('Succesfully saved story ' + story['title'])
  else:
    print('Failed to save story ' + story['title'])